Requirement:
- traitlets
- ftp
- csv
- netmiko
- paramiko
- pyserial
- ftplib
- file crede.py
- file configurasi switch

### Pilih file konfigurasi yang telah dibuat sesuai dengan kantor tujuan

In [ ]:
#!/bin/env python

import serial,sys,glob,select, io, time
import serial.tools.list_ports as port_list

import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog


class SelectFilesButton(widgets.Button):

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Pilih File"
        self.icon = "square-o"
        self.style.button_color = "red"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        with out:
            try:
                # Create Tk root
                root = Tk()
                # Hide the main window
                root.withdraw()
                # Raise the root to the top of all windows.
                root.call('wm', 'attributes', '.', '-topmost', True)
                # List of selected fileswill be set to b.value
                b.files = filedialog.askopenfilename(multiple=True)

                b.description = "Files Telah Dipilih"
                b.icon = "check-square-o"
                b.style.button_color = "blue"
            except:
                pass
out = widgets.Output()
raw = SelectFilesButton()
widgets.VBox([raw, out])

### Pastikan telah memilih  file konfigurasi dengan tombol di atas

In [ ]:
ports = list(port_list.comports())
for p in ports:
    print (p)
print("---")

#Cek serport apakah sesuai dengan port switch
serport = 'COM3'
rate = "9600"
ser = serial.Serial(
    port=serport,
    baudrate=rate,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    bytesize=serial.EIGHTBITS,
    timeout=5)

print("Serial port: " + ser.portstr + "\nBaudrate: " + rate)

if not ser.isOpen():
    ser.open()

print("Status serial connection : ", ser.is_open)

sio = io.TextIOWrapper(io.BufferedRWPair(ser, ser))

#'''
def eksekusi(perintah):
    print("Perintah :", perintah)
    #comd = "'"+perintah+r"\n"+"'"
    #print("Perintah :", comd)
    
    sio.write(str(perintah))
    time.sleep(1)
    sio.write("\n")
    #print("Flushing..")
    sio.flush()
    #print("Sleeping..")
    time.sleep(2)
    #print("Readlines..")
    if ser.inWaiting():
        data_str =ser.read(ser.inWaiting())
        print("Respon :", data_str.decode())
        #print("Stringify :", data_str)
    

        

##eksekusi tiap baris perintah di file config
#file = open('template.txt','r')
file = open(raw.files[0],'r')
lines = file.readlines()

for line in lines:
    #print("Baris dari file :", line.strip())
    eksekusi(line.strip())
    
file.close()

## show version untuk mendapatkan serial number switch
sio.write(str("show version"))
time.sleep(1)
sio.write("\n")
sio.flush()
time.sleep(2)
if ser.inWaiting():
    data_sn = ser.read(ser.inWaiting())
    print("Info :", data_sn.decode())

##write sn ke file lokal
## Masukkan nama kppn untuk nama file serial number
sn = open('sn-'+str(input("Nama KPPN : "))+'.txt','w')
sn.write(data_sn.decode())
sn.close()


##Cek IP switch untuk dilakukan SSH
sio.write(str("show arp"))
time.sleep(1)
sio.write("\n")
sio.flush()
time.sleep(2)
if ser.inWaiting():
    data_sn =ser.read(ser.inWaiting())
    #print(data_sn)
    print("Info : ", data_sn.decode())

print("Proses konfigurasi switch selesai")
print("Catat IP switch di atas untuk menjadi input proses SSH")
print("Menutup serial connection")
ser.close()

In [ ]:
## Eksekusi ini HANYA jika ada error ketika koneksi serial di cell sebelumnya error.
## serial connection harus diclose agar bisa connect kembali
ser.close()

### Koneksi SSH ke switch yang telah diconfig di atas untuk menguji SSH bisa dilakukan serta meng-copy setting running-config ke ftp PJKD

In [5]:
import os
from ftplib import FTP
import crede

device_config = {
    "device_type" : "dell_os6",
    "ip" : str(input("IP untuk SSH sesuai dengan hasil show arp di atas : ")),
    "username" : crede.username,
    "password" : crede.password,
    "secret" : crede.secret
}
kanwil = "kanwil-"+str(input("Folder : kanwil-"))
kppn = "kppn-"+str(input("Folder : kppn-"))

##cek folder kanwil dan kppn di ftp
ftp = FTP('10.100.244.240')
ftp.login(crede.ftp_user,crede.ftp_pass)
print(ftp.cwd('switch_DELL_KV/backup-konfig-kantor-vertikal/'))
try:
    ftp.mkd(kanwil)
except Exception as e:
    print("Folder kanwil tersebut telah ada")
ftp.cwd(kanwil)
try:
    ftp.mkd(kppn)
except Exception as e:
    print("Folder KPPN tersebut telah ada")
print(ftp.cwd)
ftp.close()
      
from netmiko import ConnectHandler
try:
    conn = ConnectHandler(**device_config)
    #print(conn.find_prompt())
except Exception as e:
    print(e)
    
print(conn.send_command("show version"))
output = conn.send_command_timing(
    "copy run ftp://nagios@10.100.244.240/switch_DELL_KV/backup-konfig-kantor-vertikal/"+kanwil+"/"+kppn+"/"+kppn+".txt"
)
print(output)
if "Remote Password" in output:
    output+=conn.send_command_timing(crede.ftp_pass+"\n")
print(output)
if "(y/n)" in output:
    output+=conn.send_command_timing("y\n")
print(output)
## Tampilan respon baliknya tidak utuh,
## mungkin karena timingnya kurang lama,
## tapi sudah berhasil terekskusi

## Terakhir dikonek koneksi
conn.disconnect()

IP untuk SSH sesuai dengan hasil show arp di atas : 10.100.244.175
Folder : kanwil-maluku
Folder : kppn-ambon
250 CWD command successful
Folder kanwil tersebut telah ada
Folder KPPN tersebut telah ada
<bound method FTP.cwd of <ftplib.FTP object at 0x0000029EB5785CD0>>
Machine Description............... Dell Networking Switch
System Model ID................... N1548P
Machine Type...................... Dell Networking N1548P
Serial Number..................... CN03WY51DND009CG0461A01
Manufacturer...................... 0xbc00
Burned In MAC Address............. 2004.0F6B.9612
System Object ID.................. 1.3.6.1.4.1.674.10895.3066
SOC Version....................... BCM56150_A0
HW Version........................ 3
CPLD Version...................... 16

unit active      backup      current-active next-active
---- ----------- ----------- -------------- --------------
1    6.3.3.14    6.3.3.14    6.3.3.14       6.3.3.14


copy running-config ftp://nagios@10.100.244.240/switch_DELL_KV/back